In [1]:
versioninfo()

Julia Version 0.5.0-dev+714
Commit af71b86* (2015-10-11 07:51 UTC)
Platform Info:
  System: Darwin (x86_64-apple-darwin13.4.0)
  CPU: Intel(R) Core(TM) i7-2635QM CPU @ 2.00GHz
  WORD_SIZE: 64
  BLAS: libopenblas (DYNAMIC_ARCH NO_AFFINITY Sandybridge)
  LAPACK: libopenblas
  LIBM: libopenlibm
  LLVM: libLLVM-3.3


In [2]:
type LIFSomas
    Id::Array{Float32,1} # dendritic input current
    Is::Array{Float32,1} # synaptic input current
    u::Array{Float32,1} # membrane potential
    z::Array{Int32,1} # spike count
    θ::Float32 # spiking threshold
    ρ::Float32 # reset potential
    τ::Float32 # membrane time constant
    g_leak::Float32 # leak conductance
end
Base.length(somas::LIFSomas) = length(somas.u)

function LIFSomas(n::Int; θ=0.8, ρ=0.1, τ=20e-3, g_leak=1.0)
    Id = zeros(Float32, n)
    Is = zeros(Float32, n)
    us = zeros(Float32, n)
    zs = zeros(Int32, n)
    LIFSomas(Id, Is, us, zs, θ, ρ, τ, g_leak)
end

LIFSomas

In [3]:
somas = LIFSomas(100)

LIFSomas(Float32[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],Float32[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],Float32[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],Int32[0,0,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0],0.8f0,0.1f0,0.02f0,1.0f0)

In [4]:
function update(group, dt)
    Id_group = group.Id
    Is_group = group.Is
    u_group = group.u
    z_group = group.z
    θ_group = group.θ
    ρ_group = group.ρ
    τ_group = group.τ
    g_leak_group = group.g_leak
    @simd for i = 1:length(group)
            @inbounds begin
                    I_leak = -(u_group[i]) * g_leak_group * (u_group[i] > 0)
                    dudt = (Id_group[i] + Is_group[i] + I_leak) / τ_group
                    u_group[i] = u_group[i] + dudt * dt
                end
        end
end

update (generic function with 1 method)

In [5]:
update(somas, 1e-3)

In [8]:
@time update(somas, 1e-3)

  0.000044 seconds (304 allocations: 4.844 KB)


In [9]:
@code_llvm update(somas, 1e-3)


define void @julia_update_25076(%jl_value_t*, double) {
top:
  %2 = alloca %UnitRange.0
  %3 = alloca %UnitRange.0
  %4 = alloca %UnitRange.0
  %5 = alloca %jl_value_t*, i32 9
  %6 = getelementptr %jl_value_t** %5, i32 2
  %7 = getelementptr %jl_value_t** %5, i32 6
  %8 = bitcast %jl_value_t** %5 to i64*
  store i64 14, i64* %8
  %9 = load %jl_value_t*** @jl_pgcstack
  %10 = getelementptr %jl_value_t** %5, i32 1
  %11 = bitcast %jl_value_t** %10 to %jl_value_t***
  store %jl_value_t** %9, %jl_value_t*** %11
  store %jl_value_t** %5, %jl_value_t*** @jl_pgcstack
  store %jl_value_t* null, %jl_value_t** %6
  %12 = getelementptr %jl_value_t** %6, i32 1
  store %jl_value_t* null, %jl_value_t** %12
  %13 = getelementptr %jl_value_t** %6, i32 2
  store %jl_value_t* null, %jl_value_t** %13
  %14 = getelementptr %jl_value_t** %6, i32 3
  store %jl_value_t* null, %jl_value_t** %14
  store %jl_value_t* null, %jl_value_t** %7
  %15 = getelementptr %jl_value_t** %7, i32 1
  store %jl_value_t* null

In [10]:
@code_native update(somas, 1e-3)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[4]
Source line: 4
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	subq	$216, %rsp
	movsd	%xmm0, -192(%rbp)
Source line: 4
	movq	$14, -160(%rbp)
	movabsq	$jl_pgcstack, %rcx
	movq	(%rcx), %rax
	movq	%rax, -152(%rbp)
	leaq	-160(%rbp), %rax
	movq	%rax, (%rcx)
	movq	$0, -144(%rbp)
	movq	$0, -136(%rbp)
	movq	$0, -128(%rbp)
	xorps	%xmm0, %xmm0
	leaq	-56(%rbp), %rax
Source line: 10
Source line: [inline] array.jl:56
	movabsq	$colon, %r14
Source line: 4
	movq	$0, -120(%rbp)
	movq	$0, -112(%rbp)
	movups	%xmm0, -104(%rbp)
Source line: 2
	movq	(%rdi), %rcx
	movq	%rcx, -144(%rbp)
Source line: 3
	movq	8(%rdi), %rcx
	movq	%rcx, -136(%rbp)
Source line: 4
	movq	16(%rdi), %rcx
	movq	%rcx, -128(%rbp)
Source line: 9
	movss	44(%rdi), %xmm0
Source line: 8
	movss	%xmm0, -196(%rbp)
	movss	40(%rdi), %xmm0
Source line: 10
Source line: [inline] In[2]:11
	movss	%xmm0, -200(%rbp)
	movq	16(%rdi), %rcx
	movq	%rcx,